In [133]:
import json
import time
from datetime import datetime, timedelta
from os import listdir

files = listdir('../tweet_data')
print files

from os.path import splitext
fname = splitext(files[0])[0] + '.csv'
print fname

f = open('../tweet_data/tweets_#gopatriots.txt')

import json
import pprint

['tweets_#gohawks.txt', 'tweets_#gopatriots.txt', 'tweets_#nfl.txt', 'tweets_#patriots.txt', 'tweets_#sb49.txt', 'tweets_#superbowl.txt']
tweets_#gohawks.csv


In [102]:
import re
m = re.search('#.+', splitext(files[0])[0])
if m:
    hashtag = m.group()
    print "Processing '" + hashtag + "'"

Processing '#gohawks'


In [127]:
HOUR_INDEX = 3
MINUTE_INDEX = 4
SECOND_INDEX = 5

print time.localtime(tweet['firstpost_date']).tm_hour

# # get start hour from first tweet
start_time = list(time.localtime(tweet['firstpost_date']))
# start_time[HOUR_INDEX] = 23; start_time[MINUTE_INDEX] = 0; start_time[SECOND_INDEX] = 0
# nth_hour = start_time[HOUR_INDEX]
start_time = time.mktime(start_time)
print "Starting from time %s" % time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(start_time))
    
# # time_format = "%Y-%m-%d %H:%M:%S"
# timespan = 3600
# maxtime = start_time + timespan

# print start_time
# print maxtime
# print time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(maxtime))

# timespan


15
Starting from time 2015-02-07 15:54:35


In [134]:
line = f.readline()    
tweet = json.loads(line)

n_tweets = 0 # number of tweets per hour
n_followers = {} # number of followers of users posting the tweets per hour
n_retweets = 0 # number of retweets per hour
num_window = 0 # total number of hours

user_follower = {}

# outputs
import numpy as np

"""
output: n_tweets  n_retweets  sum_followers  max_followers  hour
"""
output = np.array([])

HOUR_INDEX = 3
MINUTE_INDEX = 4
SECOND_INDEX = 5
# get start hour from first tweet
start_time = list(time.localtime(tweet['firstpost_date']))
start_time[MINUTE_INDEX] = 0; start_time[SECOND_INDEX] = 0
nth_hour = start_time[HOUR_INDEX]
start_time = time.mktime(start_time)
print "Starting from time %s" % time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(start_time))
    
time_format = "%Y-%m-%d %H:%M:%S"

while len(line) != 0:
    tweet = json.loads(line)
    line = f.readline()
    tweet_time = tweet['firstpost_date']
    tweet_hour = time.localtime(tweet_time).tm_hour
    print "tweet time %s" % time.strftime(time_format, time.localtime(tweet_time))
    if start_time < tweet_time and tweet_hour != nth_hour:
        print "Finished processing %d tweets from hour %d, next date time %s" %\
              (n_tweets, nth_hour, time.strftime(time_format, time.localtime(tweet_time)))
        # update output
        new_hour = [n_tweets, n_retweets, sum(n_followers), max(n_followers), nth_hour]
        np.append(output, new_hour, axis=0)
        # clear & update hourly variables
        num_window += 1
        n_tweets = 0 # number of tweets per hour
        n_followers = {} # number of followers of users posting the tweets per hour
        n_retweets = 0 # number of retweets per hour
        nth_hour = tweet_hour  # hour of the day
    # extract features
    n_tweets += 1
    n_retweets += tweet['tweet']['retweet_count']
    n_followers[tweet['tweet']['user']['id']] = tweet['tweet']['user']['followers_count']
    user_follower[tweet['tweet']['user']['id']] = tweet['tweet']['user']['followers_count']
    start_time = tweet_time
    
print "Average number of tweets per hour %f" % np.mean(output[:, 0])
print "Average number of retweets %f" % np.sum(output[:, 1]) / float(np.sum(output[:, 0]))
print "Average number of followers of users %f" % sum(user_follower.values()) / float(len(user_follower))
                      
f.close()

Starting from time 2015-01-10 04:00:00
ntweets 0
ntweets 0
tweet time 2015-01-10 04:30:45
ntweets 0
ntweets 1
ntweets 1
tweet time 2015-01-11 03:19:12
ntweets 1
Finished processing 1 tweets 4-th hour, next date time 2015-01-11 03:19:12
ntweets 0
ntweets 1
ntweets 1
tweet time 2015-01-11 08:11:20
ntweets 1
Finished processing 1 tweets 3-th hour, next date time 2015-01-11 08:11:20
ntweets 0
ntweets 1
ntweets 1
tweet time 2015-01-11 08:44:05
ntweets 1
ntweets 2
ntweets 2
tweet time 2015-01-13 06:58:22
ntweets 2
Finished processing 2 tweets 8-th hour, next date time 2015-01-13 06:58:22
ntweets 0
ntweets 1
ntweets 1
tweet time 2015-01-13 08:46:10
ntweets 1
Finished processing 1 tweets 6-th hour, next date time 2015-01-13 08:46:10
ntweets 0
ntweets 1
ntweets 1
tweet time 2015-01-14 08:57:58
ntweets 1
ntweets 2
ntweets 2
tweet time 2015-01-14 10:20:17
ntweets 2
Finished processing 2 tweets 8-th hour, next date time 2015-01-14 10:20:17
ntweets 0
ntweets 1
ntweets 1
tweet time 2015-01-14 17:50:

KeyboardInterrupt: 